In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///recom-films.db')
Session = sessionmaker(bind=engine)
session = Session()

In [2]:
query = text("""
    SELECT m.id, m.title, m.synopsis, m.year, m.month, m.day,
           GROUP_CONCAT(DISTINCT g.name) as genres,
           GROUP_CONCAT(DISTINCT CASE WHEN p.isRealisator = 1 THEN p.name END) as directors,
           GROUP_CONCAT(DISTINCT CASE WHEN p.isActor = 1 THEN p.name END) as actors
    FROM movie m
    LEFT JOIN movie_genre mg ON m.id = mg.movie_id
    LEFT JOIN genre g ON mg.genre_id = g.id
    LEFT JOIN movie_participant mp ON m.id = mp.movie_id
    LEFT JOIN participant p ON mp.participant_id = p.id
    GROUP BY m.id
""")

result = session.execute(query)
movies_data = result.fetchall()
movies_data

[(1, 'Aguirre, la colère de Dieu', "En 1560, Aguirre, un conquistador illuminé, s'enfonce avec ses hommes au coeur de la forêt vierge amazonienne, à la recherche de l'Eldorado.", 1975, 2, 26, 'Action,Aventure,Drame', None, 'Klaus Kinski,Ruy Guerra,Helena Rojo,Werner Herzog'),
 (2, 'Superman', 'Il est partout ! Delaissant son enveloppe de reporter timide, Kent se transforme en Superman. Il vole au secours des avions en detresse, plonge dans les entrailles de la terre, et affronte un dangereux savant voulant dominer le monde.', 1979, 1, 26, 'Action,Aventure,Science Fiction', 'Richard Donner', 'Christopher Reeve,Marlon Brando,Gene Hackman'),
 (3, 'La Balade sauvage', "Inspirée par l'histoire authentique de Charlie Stark-Weather, jeune délinquant des années cinquante, évocation de la folle équipée de deux jeunes ama ... (25 characters truncated) ...  droit de s'aimer. Ils laissent sur leur passage de nombreux cadavres dont le père de la jeune fille, qui refusait que celle-ci fréquente un é

In [3]:
import pandas as pd

movies_df = pd.DataFrame(movies_data, columns=['id', 'title', 'synopsis', 'year', 'month', 'day', 'genres', 'realisator', 'actors'])

movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22868 entries, 0 to 22867
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          22868 non-null  int64 
 1   title       22868 non-null  object
 2   synopsis    22868 non-null  object
 3   year        22868 non-null  int64 
 4   month       22868 non-null  int64 
 5   day         22868 non-null  int64 
 6   genres      22420 non-null  object
 7   realisator  19644 non-null  object
 8   actors      22420 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.6+ MB


In [4]:
# On affiche les valeurs manquantes
movies_df.isnull().sum()

id               0
title            0
synopsis         0
year             0
month            0
day              0
genres         448
realisator    3224
actors         448
dtype: int64

In [5]:
movies_df = movies_df.dropna()
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19220 entries, 1 to 22867
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          19220 non-null  int64 
 1   title       19220 non-null  object
 2   synopsis    19220 non-null  object
 3   year        19220 non-null  int64 
 4   month       19220 non-null  int64 
 5   day         19220 non-null  int64 
 6   genres      19220 non-null  object
 7   realisator  19220 non-null  object
 8   actors      19220 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.5+ MB


In [6]:
# remplace les décimales par des entiers
movies_df.year = movies_df.year.astype(int)
movies_df.month = movies_df.month.astype(int)
movies_df.day = movies_df.day.astype(int)

In [7]:
def create_soup(x):
    features = []
    if isinstance(x['genres'], str):
        features.extend(x['genres'].split(','))
    if isinstance(x['realisator'], str):
        features.extend(x['realisator'].split(','))
    if isinstance(x['actors'], str):
        features.extend(x['actors'].split(','))
    if isinstance(x['synopsis'], str):
        features.extend(x['synopsis'].split())
    return ' '.join(features)

movies_df['soup'] = movies_df.apply(create_soup, axis=1)

In [8]:
import numpy as np
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer

# Supposons que movies_df est déjà défini et contient les colonnes 'soup' et 'title'

# Initialisation du vectoriseur TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Création de la matrice TF-IDF
tfidf_matrix = tfidf.fit_transform(movies_df['soup'])

# Conversion en tableau numpy pour FAISS
tfidf_array = tfidf_matrix.toarray().astype('float32')

# Création de l'index FAISS
dimension = tfidf_array.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(tfidf_array)

In [9]:

def get_recommendations(query, top_n=10):
    query_tfidf = tfidf.transform([query]).toarray().astype('float32')
    D, I = index.search(query_tfidf, top_n)
    return [(movies_df['title'].iloc[i], float(D[0][j])) for j, i in enumerate(I[0])]

def search_recommendations(genres=None, director=None, actors=None, top_n=10):
    query_parts = []
    if genres:
        if isinstance(genres, str):
            query_parts.append(genres)
        elif isinstance(genres, list):
            query_parts.extend(genres)
    if director:
        query_parts.append(director)
    if actors:
        if isinstance(actors, str):
            query_parts.append(actors)
        elif isinstance(actors, list):
            query_parts.extend(actors)
    
    query = ', '.join(query_parts)
    return get_recommendations(query, top_n)

In [10]:
results = search_recommendations(genres="Action", director="Christopher Nolan", actors=["Chrisian Bale"], top_n=10)
for title, score in results:
    print(f"{title}: {score}")

Oppenheimer: 0.4250968098640442
The Dark Knight, Le Chevalier Noir: 0.3384535312652588
Batman Begins: 0.3209243416786194
Le Prestige: 0.29220572113990784
The Dark Knight Rises: 0.27923211455345154
Dunkerque: 0.2283947467803955
Following, le suiveur: 0.21603509783744812
Inception: 0.19881471991539001
Memento: 0.19529014825820923
Interstellar: 0.18777646124362946


In [11]:
# On va sauvegarder le vectoizer et l'index pour les réutiliser plus tard
import pickle

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Puis on sauvegarde l'index 
faiss.write_index(index, 'faiss_index.index')

In [12]:
# On sauvegarde aussi le DataFrame dans un fichier pickle
movies_df.to_pickle('movies_df.pkl')

In [13]:
# On peut charger le vectorizer et l'index
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

index = faiss.read_index('faiss_index.index')

# On peut maintenant réutiliser les fonctions get_recommendations et search_recommendations
results = search_recommendations(genres="Action", director="Christopher Nolan", actors=["Chrisian Bale"], top_n=10)
for title, score in results:
    print(f"{title}: {score}")

Oppenheimer: 0.4250968098640442
The Dark Knight, Le Chevalier Noir: 0.3384535312652588
Batman Begins: 0.3209243416786194
Le Prestige: 0.29220572113990784
The Dark Knight Rises: 0.27923211455345154
Dunkerque: 0.2283947467803955
Following, le suiveur: 0.21603509783744812
Inception: 0.19881471991539001
Memento: 0.19529014825820923
Interstellar: 0.18777646124362946
